In [1]:
from hyper import *
from alignment import *
from datasets.preprecossing import *
from core import *
from datasets.loading import *
from datasets.hc_dataset import *
from datasets.balance_dataset import *
from utils.linkage import *
from model.balancehc import balancehc

from utils.poincare import *
import scib
import shutil



In [2]:
import os
import numpy as np
import pandas as pd
import scib

import scanpy as sc
from pathlib import Path
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from os.path import exists
def get_count_data(adata,counts_location=None):
    
    data = adata.layers[counts_location].copy() if counts_location else adata.X.copy()
    if not isinstance(data, np.ndarray):
        data= data.toarray()
    data_df = pd.DataFrame(data,index=adata.obs_names,columns=adata.var_names).transpose()
    return data_df


def check_paths(output_folder,output_prefix=None):
    # Create relative path
    output_path = os.path.join(os.getcwd(), output_folder)

    # Make sure that the folder exists
    Path(output_path).mkdir(parents=True, exist_ok=True)

    if os.path.exists(os.path.join(output_path, f"{output_prefix}assigned_locations.csv")):
        print("\033[91mWARNING\033[0m: Running this will overwrite previous results, choose a new"
              " 'output_folder' or 'output_prefix'")

    return output_path
def remove_batch_effect(pseudo_bulk, bulk_adata, out_dir, project='',batch_effect=True):
    """
    Remove batch effect between pseudo_bulk and input bulk data.

    Parameters
    ----------
    pseudo_bulk : anndata.AnnData
        An :class:`~anndata.AnnData` containing the pseudo expression.
    bulk_adata : anndata.AnnData
        An :class:`~anndata.AnnData` containing the input expression.
    out_dir : string, optional
        The path to save the output file.
    project : string, optional
        The prefix of output file.
        
    Returns
    -------
    Returns the expression after removing batch effect.

    """
    out_dir = check_paths(out_dir+'/batch_effect')
    if batch_effect:
        if exists(f'{out_dir}/{project}_batch_effected.txt'):
            print(f'{out_dir}/{project}_batch_effected.txt already exists, skipping batch effect.')
            bulk_data = pd.read_csv(f"{out_dir}/{project}_batch_effected.txt",sep='\t').T
        else:
            
            save=True
            # check path, file will be stored in out_dir+'/batch_effect'
            pseudo_bulk_df = get_count_data(pseudo_bulk)
            input_bulk_df= get_count_data(bulk_adata)

            bulk = pd.concat([pseudo_bulk_df,input_bulk_df], axis=1)

            cells = np.append(pseudo_bulk.obs_names, bulk_adata.obs_names)
            batch = np.append(np.ones((1,len(pseudo_bulk.obs_names))), np.ones((1,len(bulk_adata.obs_names)))+1)
            if save:
                bulk.to_csv(out_dir+f"/{project}_before_batch_effected.txt",sep='\t')
            meta = pd.DataFrame({"batch": batch,"cells":cells})
            # get r script path
            robjects.r.source('./combat.R')
            pandas2ri.activate()
            robjects.r.run_combat(bulk, meta, out_dir, project)
            # stop auto trans from pandas to r dataframe
            pandas2ri.deactivate()
            # add layer
            if exists(f'{out_dir}/{project}_batch_effected.txt'):
                bulk_data = pd.read_csv(f"{out_dir}/{project}_batch_effected.txt",sep='\t').T
                print(bulk_data.shape)
            else:
                raise ValueError('The batch_effected data is not available')
        bulk_data.clip(lower=0,inplace=True)
        # print(pseudo_bulk)
        # print(pseudo_bulk.obs_names)
        pseudo_bulk.layers["batch_effected"] = bulk_data.loc[pseudo_bulk.obs_names,:].values
        bulk_adata.layers["batch_effected"] = bulk_data.loc[bulk_adata.obs_names,:].values
    else:
        pseudo_bulk_df = get_count_data(pseudo_bulk)
        input_bulk_df= get_count_data(bulk_adata)
        bulk = pd.concat([pseudo_bulk_df,input_bulk_df], axis=1)
        bulk.to_csv(out_dir+f"/{project}_batch_effected.txt",sep='\t')

    return pseudo_bulk,bulk_adata

In [3]:
cell_path1 = './datas/32/1/BRCA_GSE110686.h5ad'
folder_path1 = './datas/32/1/'
radius1 = 0
c1 =0.1
epoches1 =40
cell_path2 = "./datas/32/2/8000_sample.h5ad" 
folder_path2 = "./datas/32/2/" 
radius2 = 0
c2 =0.1
epoches2 = 40
contin = False
resolution=0.5
method='average'
alignment=1
n_pca=100
meta_col = 'Celltype..major.lineage.'


In [4]:
# python run_sc.py -cp1 './datas/120/3/sample1_small.h5' -f1 "./datas/120/3/" -r1 25 -c1 0.1 -e1 10 -cp2 './datas/120/4/sample1_small.h5' -f2 "./datas/120/4/" -r2 25 -c2 0.1 -e2 10 --contin False --alignment 1 --resolution 1 --n_pca 500


In [5]:
if (contin==False) or ((os.path.exists(folder_path1+'merge_cell_data.csv') and os.path.exists(folder_path1 + 'merge_cell_meta.csv')) == False):
    loss1 = merge_by_radius(cell_path1,folder_path1,radius1,method,meta_col)
    print("cell meta score for dataset1: {}\n".format(loss1))
else:
    print("dataset1 find files and skip merging")


adata1 = pd.read_csv(folder_path1+"merge_cell_data.csv")
cell_meta = pd.read_csv(folder_path1+"merge_cell_meta.csv")
cell_meta = cell_meta.set_index(cell_meta.columns[0])
adata1 = adata1.set_index(adata1.columns[0])
adata1 = anndata.AnnData(adata1)
adata1.obs['celltype'] = cell_meta.values.reshape(-1)


if(contin==False) or ((os.path.exists(folder_path2+'merge_cell_data.csv') and os.path.exists(folder_path2 + 'merge_cell_meta.csv')) == False):
    loss2 = merge_by_radius(cell_path2,folder_path2,radius2,method,meta_col)
    print("cell meta score for dataset2: {}".format(loss2))
else:
    print("dataset2 find files and skip merging")

adata2 = pd.read_csv(folder_path2+"merge_cell_data.csv")
cell_meta = pd.read_csv(folder_path2+"merge_cell_meta.csv")
cell_meta = cell_meta.set_index(cell_meta.columns[0])
adata2 = adata2.set_index(adata2.columns[0])
adata2 = anndata.AnnData(adata2)
adata2.obs['celltype'] = cell_meta.values.reshape(-1)



preprocessing_cluster(adata1,N_pcs=n_pca,resolution=resolution)
preprocessing_cluster(adata2,N_pcs=n_pca,resolution=resolution)

inter_gene = sort_data(adata1,adata2)

tmp1 = calculate_cluster_centroid_for_genes(adata1,inter_gene,folder_path1)
tmp2 = calculate_cluster_centroid_for_genes(adata2,inter_gene,folder_path2)

ari = adjusted_rand_score(adata1.obs['celltype'].tolist(), adata1.obs['leiden'].tolist())
print("ARI score for adata1: ", ari)

ari = adjusted_rand_score(adata2.obs['celltype'].tolist(), adata2.obs['leiden'].tolist())
print("ARI score for adata2: ", ari)

meta_list1 = []
clustername = adata1.obs['leiden'].unique().tolist()
clustername = list(map(int, clustername))
clustername.sort()
for value in clustername:
    indices = [i for i, x in enumerate(adata1.obs['leiden']) if x == str(value)]
    t = [adata1.obs['celltype'].tolist()[index] for index in indices]
    most_common_element = max(t, key=t.count)
    meta_list1.append(most_common_element)
np.save(folder_path1+'tree_merge.npy',meta_list1)

    
meta_list2 = []
clustername = adata2.obs['leiden'].unique().tolist()
clustername = list(map(int, clustername))
clustername.sort()
for value in clustername:
    indices = [i for i, x in enumerate(adata2.obs['leiden']) if x == str(value)]
    t = [adata2.obs['celltype'].tolist()[index] for index in indices]
    most_common_element = max(t, key=t.count)
    meta_list2.append(most_common_element)
np.save(folder_path2+'tree_merge.npy',meta_list2)


v1 = pd.read_csv(folder_path1+"merge_labels.csv")
meta = pd.read_csv(folder_path1+"merge_cell_meta.csv")
meta = meta.set_index(meta.columns[0])
meta
lisan = []
julei = []
for i in range(len(v1)):
    lisan.append(meta.iloc[v1['label'][i]][0])
    julei.append(adata1.obs['leiden'].iloc[v1['label'][i]][0])
v1['first']=lisan
v1['second']=julei
v1.to_csv(folder_path1+'meta_result.csv',index=None)

v1 = pd.read_csv(folder_path2+"merge_labels.csv")
meta = pd.read_csv(folder_path2+"merge_cell_meta.csv")
meta = meta.set_index(meta.columns[0])
meta
lisan = []
julei = []
for i in range(len(v1)):
    lisan.append(meta.iloc[v1['label'][i]][0])
    julei.append(adata2.obs['leiden'].iloc[v1['label'][i]][0])
v1['first']=lisan
v1['second']=julei
v1.to_csv(folder_path2+'meta_result.csv',index=None)

if(contin==False) or ((os.path.exists(folder_path1 + 'dataxy.npy') and os.path.exists(folder_path1+'data1link.npy') and os.path.exists(folder_path1+'dataname.npy')) == False):
    get_Hyper_tree(folder_path1+'datas.data',1,tmp1.shape[1]+1,0,epoches1,save_path=folder_path1)
else:
    print("dataset1 find files and skip embedding");

if(contin==False) or ((os.path.exists(folder_path2 + 'dataxy.npy') and os.path.exists(folder_path2+'data1link.npy') and os.path.exists(folder_path1+'dataname.npy')) == False):
    get_Hyper_tree(folder_path2+'datas.data',1,tmp2.shape[1]+1,0,epoches2,save_path=folder_path2)
else:
    print("dataset2 find files and skip embedding")

    
nodes1,n1 = build_hyper_tree_from_folder(folder_path1)
nodes2,n2 = build_hyper_tree_from_folder(folder_path2)

merge_list1 = [];
merge_list2 = [];
nodes1[0] = search_tree(nodes1[0],c1,merge_list1)
nodes2[0] = search_tree(nodes2[0],c2,merge_list2)

for i in range(len(nodes1)):
    if(int(nodes1[i].name)<len(meta_list1)):
        nodes1[i].name= nodes1[i].name +'_'+ meta_list1[int(nodes1[i].name)];
        
for i in range(len(nodes2)):
    if(int(nodes2[i].name)<len(meta_list2)):
        nodes2[i].name= nodes2[i].name +'_'+ meta_list2[int(nodes2[i].name)];  
        
rate = 0;        
if(alignment==1):
    rate,anslist,ans = run_alignment(nodes1,nodes2,folder_path1,folder_path2,meta_list1,meta_list2);
elif(alignment==2):
    rate = run_alignment_linear(nodes1,nodes2);
    
rate

/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/orig.ident' was written without encoding metadata.
  return read_elem(dataset)
/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/nCount_RNA' was written without encoding metadata.
  return read_elem(dataset)
/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/nFeature_RNA' was written without encoding metadata.
  return read_elem(dataset)
/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/UMAP_1' was written without encoding metadata.
  return read_elem(dataset)
/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/UMAP_2' was written without encoding metadata.
  return 

cell meta score for dataset1: 1.0



/var/folders/bm/69jvgsdj3rxch558kx9yp9y00000gn/T/ipykernel_40421/2407986756.py:12: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata1 = anndata.AnnData(adata1)
/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/scanpy/preprocessing/_highly_variable_genes.py:208: RuntimeWarning: invalid value encountered in log
  dispersion = np.log(dispersion)
100%|██████████████████████████████████| 8000/8000 [00:00<00:00, 7752872.46it/s]


In [ ]:
anslist_dist = dict(ans)
anslist_dist.keys()
def search_lineage(now,path,anss):
    path.append(now.name)
    if(now.son==[]):
        anss.append(path);
        return
    
    for i in now.son:
        search_lineage(i,path.copy(),anss);
temp1 = []
search_lineage(nodes1[0],[],temp1)
temp1

In [ ]:
route1 = []
route2 = []

for i in temp1:
    r1 = []
    r2 = []
    for j in i:
        if j in anslist_dist.keys():
            r1.append(j)
            r2.append(anslist_dist[j])
    route1.append(r1)
    route2.append(r2)
route2,route1


In [ ]:
adata1.obs.index = [i+'_1' for i in adata1.obs.index]
adata2.obs.index = [i+'_2' for i in adata2.obs.index]

In [ ]:
score = [];


for i,j in zip(route1,route2):
    try:
        shutil.rmtree('./batch_effect/', ignore_errors=True)

        cells1 = [ ]
        cells2 = [ ]
        for k,t in zip(i,j):
            num1 = int(k.split('_')[0])
            num2 = int(t.split('_')[0])
            if(num1 < (n1+1)/2 and num2 < (n2+1)/2):
                # cells1.append(num1)
                # cells2.append(num2)
                cells1.append(str(num1))
                cells2.append(str(num2))
        if(cells1==[] or cells2 ==[]):
            continue;
        
        sub_adata1 = adata1[adata1.obs['leiden'].isin(cells1),inter_gene].copy();
        sub_adata2 = adata2[adata2.obs['leiden'].isin(cells2),inter_gene].copy();

        adata1_after,adata2_after = remove_batch_effect(sub_adata1.copy(),sub_adata2.copy(),'./')
        
        sc.pp.neighbors(sub_adata1,use_rep='X')
        sc.tl.diffmap(sub_adata1)
        sub_adata1.uns['iroot'] = 0
        sc.tl.dpt(sub_adata1)
        
        
        sc.pp.neighbors(sub_adata2,use_rep='X')
        sc.tl.diffmap(sub_adata2)
        sub_adata2.uns['iroot'] = 0
        sc.tl.dpt(sub_adata2)
        
        
        adata1_after.obsm['batch_effected'] = adata1_after.layers['batch_effected']
        adata2_after.obsm['batch_effected'] = adata2_after.layers['batch_effected']
        
        sc.pp.neighbors(adata1_after,use_rep='batch_effected')
        sc.tl.diffmap(adata1_after)
        adata1_after.uns['iroot'] = 0
        sc.tl.dpt(adata1_after)

        sc.pp.neighbors(adata2_after,use_rep='batch_effected')
        sc.tl.diffmap(adata2_after)
        adata2_after.uns['iroot'] = 0
        sc.tl.dpt(adata2_after)
        
        score.append( scib.me.trajectory_conservation(sub_adata1, adata1_after, label_key="celltype"))
        score.append( scib.me.trajectory_conservation(sub_adata2, adata2_after, label_key="celltype"))
        shutil.rmtree('./batch_effect/', ignore_errors=True)
    except:
        print(cells1,cells2)
        pass;
score

In [ ]:
np.array(score).mean()

In [ ]:
0.7413529924699612 啥也没有
0.6

In [ ]:
0.7413529924699612


In [ ]:
0.7554990228993519 houmian